In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
df = pd.read_csv('2012-18_playerBoxScore_diff.csv')

In [2]:
pd.set_option('display.max_columns', 999)

In [3]:
df.describe(include='all')

,Unnamed: 0,playDispNm,next_gmDays,pre_gmDays,teamDayOff,gmDay,gmDayofweek,gmDayofyear,gmElapsed,teamConfIsEast,opptConfIsEast,teamRsltIsWin,opptRsltltIsWin,teamLocIsHome,opptLocIsHome,playPos_C,playPos_F,playPos_G,playPos_PF,playPos_PG,playPos_SF,playPos_SG,teamDiv_Atlantic,teamDiv_Central,teamDiv_Northwest,teamDiv_Pacific,teamDiv_Southeast,teamDiv_Southwest,opptDiv_Atlantic,opptDiv_Central,opptDiv_Northwest,opptDiv_Pacific,opptDiv_Southeast,opptDiv_Southwest,pre_gm_miles,gmTime_diff,playMin_diff,playHeight_diff,playWeight_diff,playPTS_diff,playAST_diff,playTO_diff,playSTL_diff,playBLK_diff,playPF_diff,playFGA_diff,playFGM_diff,play2PA_diff,play2PM_diff,play3PA_diff,play3PM_diff,playFTA_diff,playFTM_diff,playORB_diff,playDRB_diff,playTRB_diff,playFG%_diff,play2P%_diff,play3P%_diff,playFT%_diff,opptDayOff_diff,gmYear_diff,gmMonth_diff,gmWeek_diff,gmIs_month_end_diff,gmIs_month_start_diff,gmIs_quarter_end_diff,gmIs_quarter_start_diff,gmIs_year_end_diff,gmIs_year_start_diff,playBYear_diff,playBMonth_diff,playBWeek_diff,playBDay_diff,playBDayofweek_diff,playBDayofyear_diff,playBIs_month_end_diff,playBIs_month_start_diff,playBIs_quarter_end_diff,playBIs_quarter_start_diff,playBIs_year_end_diff,playBIs_year_start_diff,playBElapsed_diff,IsStarter_diff,age_diff,Seats_diff,Year_Open_diff
count,154014.000000,154014,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,1.540140e+05,154014.000000,154014.000000,154014.000000,154014.000000,154014.0,154014.0,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.0,154014.000000,154014.0,154014.0,154014.0,1.540140e+05,154014.000000,154014.000000,154014.000000,154014.000000
unique,NaN,895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Corey Brewer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,77006.500000,NaN,2.349851,2.390562,2.076396,15.288493,3.008726,165.169192,1.437743e+09,0.496254,0.499435,0.591686,0.408314,1.0,0.0,0.161362,0.001078,0.000649,0.214383,0.212461,0.192125,0.217941,0.166764,0.164154,0.166556,0.167141,0.165336,0.170049,0.166323,0.166478,0.166972,0.167829,0.166634,0.165764,1561.301450,-0.033406,0.004019,-0.000013,-0.000143,0.004837,0.001643,0.000084,0.000162,-0.000467,-0.000279,0.002513,0.001786,-0.000214,0.0

In [4]:
df.drop(columns='Unnamed: 0', inplace=True)

In [5]:
X = np.array(df.drop(['playMin_diff','playDispNm'], axis=1))
y = np.array(df['playMin_diff'])

In [6]:
X.shape

(154014, 84)

In [7]:
y.shape

(154014,)

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [9]:
X_train.shape

(98568, 84)

In [10]:
X_test.shape

(30803, 84)

In [11]:
X_val.shape

(24643, 84)

### 1. Sampling the Features

In [12]:
from sklearn.ensemble import RandomForestRegressor

In [13]:
def feature_sampling_scores (max_features_list):
    feature_sampling_scores_list = []
    for max_feature in max_features_list:
        rfr = RandomForestRegressor(max_features=max_feature, random_state=1,n_estimators=10)
        rfr.fit(X_train, y_train)
        score = (max_feature, rfr.score(X_val,y_val))
        feature_sampling_scores_list.append(score)
    return feature_sampling_scores_list

In [14]:
%%time
feature_sampling_scores([0.5,'log2','sqrt',None])

CPU times: user 30.5 s, sys: 200 ms, total: 30.7 s
Wall time: 30.7 s


[(0.5, 0.7421187867265651),
 ('log2', 0.7220955233039497),
 ('sqrt', 0.732911642386389),
 (None, 0.7393985160442711)]

We can see from above that using None and 0.5 perform the best.

### 2. Number of Trees

In [15]:
rfr_n_trees = RandomForestRegressor(n_estimators=50, random_state=1, max_features=0.5)

In [16]:
%%time
rfr_n_trees.fit(X_train, y_train)

CPU times: user 45.9 s, sys: 174 ms, total: 46.1 s
Wall time: 46.2 s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=0.5, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=50,
                      n_jobs=None, oob_score=False, random_state=1, verbose=0,
                      warm_start=False)

In [17]:
rfr_n_trees.score(X_val,y_val)

0.7771546905255926

In [18]:
tree_predictions = np.vstack([estimator.predict(X_val) for estimator in rfr_n_trees.estimators_])

In [19]:
tree_predictions.shape

(50, 24643)

In [20]:
from sklearn.metrics import r2_score
r2_scores = [r2_score(y_val, np.mean(tree_predictions[:i + 1], axis = 0)) for i in range(0, len(rfr_n_trees.estimators_)) ]

In [21]:
from graph import trace_values, plot
x_vals = list(range(1, len(rfr_n_trees.estimators_) + 1))
trace = trace_values(x_vals, r2_scores)
plot([trace])

### 3. Number of leaves

In [22]:
min_samples = np.arange(2,51,1)
min_samples

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])

In [23]:
def num_leaves_wrapped_scores (min_samples):
    num_leaves_wrapped_scores_list = []
    for min_sample in min_samples:
        rfr = RandomForestRegressor(n_estimators=50, min_samples_leaf=min_sample, random_state=1)
        rfr.fit(X_train, y_train)
        score = rfr.score(X_val,y_val)
        num_leaves_wrapped_scores_list.append(score)
    return num_leaves_wrapped_scores_list

In [24]:
%%time
scores = num_leaves_wrapped_scores(min_samples)

CPU times: user 48min 32s, sys: 9.39 s, total: 48min 41s
Wall time: 48min 50s


In [25]:
scores[:5]

[0.7610242649442236,
 0.7469759432844767,
 0.7332194546281866,
 0.7193757007211421,
 0.7065639174542068]

In [26]:
wrapped_scores = np.column_stack((min_samples, scores))

In [27]:
trace_1 = trace_values(wrapped_scores[:, 0], wrapped_scores[:, 1])

In [28]:
plot([trace_1])

In [29]:
X_val.shape

(24643, 84)

In [30]:
combined_X = np.vstack((X_train, X_val))

In [31]:
combined_y = np.concatenate((y_train, y_val))

In [32]:
rfr3 = RandomForestRegressor(n_estimators=50, random_state=1, max_features=0.5)
rfr3.fit(combined_X, combined_y)
rfr3.score(X_test,y_test)

0.8298570144865287